# Artificial Neural Network: From Scratch!

## Import Libraries

In [19]:
import pandas as pd
from tensorfio.Activation import sigmoid, relu, softmax
from tensorfio.Layer import Dense
from tensorfio.Sequential import Sequential

## Import Dataset
Using the breast cancer dataset for demonstration.

In [20]:
# Import breast cancer dataset
from sklearn.datasets import load_breast_cancer

breast_cancer_data = load_breast_cancer()
df = pd.DataFrame(breast_cancer_data.data, columns=breast_cancer_data.feature_names)
df['target'] = pd.Series(breast_cancer_data.target)
df

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,target
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.16220,0.66560,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.12380,0.18660,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.14440,0.42450,0.4504,0.2430,0.3613,0.08758,0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.20980,0.86630,0.6869,0.2575,0.6638,0.17300,0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.13740,0.20500,0.4000,0.1625,0.2364,0.07678,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,21.56,22.39,142.00,1479.0,0.11100,0.11590,0.24390,0.13890,0.1726,0.05623,...,26.40,166.10,2027.0,0.14100,0.21130,0.4107,0.2216,0.2060,0.07115,0
565,20.13,28.25,131.20,1261.0,0.09780,0.10340,0.14400,0.09791,0.1752,0.05533,...,38.25,155.00,1731.0,0.11660,0.19220,0.3215,0.1628,0.2572,0.06637,0
566,16.60,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.05302,0.1590,0.05648,...,34.12,126.70,1124.0,0.11390,0.30940,0.3403,0.1418,0.2218,0.07820,0
567,20.60,29.33,140.10,1265.0,0.11780,0.27700,0.35140,0.15200,0.2397,0.07016,...,39.42,184.60,1821.0,0.16500,0.86810,0.9387,0.2650,0.4087,0.12400,0


In [21]:
df['target'].value_counts()

target
1    357
0    212
Name: count, dtype: int64

## Split Training and Test Set

In [22]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df[breast_cancer_data.feature_names], df['target'], test_size=0.2, random_state=42)

In [23]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(455, 30) (114, 30) (455,) (114,)


## Scale the Dataset

In [24]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# Implement Artificial Neural Network

## Layer

In [25]:
# class Layer:
#     '''
#     Base class for layers in the network.

#     Arguments:
#         units: Number of neurons in the layer

#     Attributes:
#         units: Number of neurons in the layer
#         built: Whether the layer has been built
#         _build_input_shape: Shape of the input to the layer
#     '''
#     def __init__(self):
#         self.built = False

#     def build(self, input_shape):
#         self._build_input_shape = input_shape
#         self.built = True

#     def call(self, inputs, *args, **kwargs):
#         return inputs
    
#     def compute_output_shape(self, input_shape):
#         return input_shape[:-1] + (self.units,)
    
#     def add_weight(self, shape):
#         # Initializes to random normal distribution
#         return np.random.normal(size=shape)

In [26]:
# class Dense(Layer):
#     '''
#     A fully-connected layer.

#     Methods:
#         build(input_shape): Builds the layer by initializing weights and biases.
#         call(inputs): Forward propagates inputs through this layer.

#     Attributes:
#         units: Number of neurons in this layer.
#         activation: Activation function to use.
#         use_bias: Whether to use a bias vector.
#         input_shape: Shape of the input tensor.

#         w: Weights of this layer.
#         b: Biases of this layer.
#         input: Input tensor.
#         z: Weighted sum of inputs.
#         a: Activation of weighted sum of inputs.
#     '''
#     def __init__(self, units, activation=None, use_bias=True, input_shape=None, **kwargs):
#         super().__init__(**kwargs)
#         self.units = units
#         self.activation = activation
#         self.use_bias = use_bias
#         self.input_shape = input_shape
    
#     def build(self, input_shape):
#         # Initialize weights and biases
#         self.w = self.add_weight([input_shape[-1], self.units]) # Shape: (input_dim, output_dim)
#         if self.use_bias:
#             self.b = self.add_weight([self.units]) # Shape: (output_dim,)
        
#         super().build(input_shape)
    
#     def call(self, inputs):
#         # Forward propagate inputs through this layer
#         # y = x * w + b
#         self.input = inputs
#         y = tf.matmul(inputs, self.w)
#         if self.use_bias:
#             y = y + self.b
#         self.z = y
#         if self.activation is not None:
#             y = self.activation(y)
#         self.a = y

#         return y


## Activation Function

In [27]:
# # Sigmoid activation function
# def sigmoid(x, derivative=False):
#     if derivative:
#         return sigmoid(x) * (1 - sigmoid(x))
    
#     return 1 / (1 + np.exp(-x))

# # ReLU activation function
# def relu(x, derivative=False):
#     if derivative:
#         return np.where(x <= 0, 0, 1)
    
#     return np.maximum(0, x)

# # Softmax activation function
# def softmax(x, derivative=False):
#     if derivative:
#         s = softmax(x)
#         return s * (1 - s)
    
#     exp = np.exp(x)
#     return exp / np.sum(exp, axis=-1, keepdims=True)

## Sequential Model

In [28]:
# class Sequential:
#     '''
#     A sequential model.

#     Methods:
#         add(layer): Adds a layer to the model.
#         build(input_shape): Builds the model by initializing weights and biases.
#         call(inputs): Forward propagates inputs through this model.
#         summary(): Prints a summary of the model.
#         compile(loss, optimizer): Compiles the model for training.
#         compute_loss(y_true, y_prob): Computes the loss between the true labels and predictions.
#         compute_metrics(y_true, y_pred): Computes the metrics for this model.
#         forward_propagation(X, y): Forward propagates the inputs and computes the loss.
#         backward_propagation(X, y, y_prob): Backward propagates the loss.
#         gradient_descent(X, y, y_true, y_prob): Performs gradient descent.
#         loss_gradient(y_true, y_prob): Computes the gradient of the loss function.
#         update_weights(grad_w, grad_b): Updates the weights and biases.
#         fit(X, y, epochs, batch_size): Trains the model.
#         predict(X): Predicts the labels for the given data.
#         evaluate(X, y): Evaluates the model on the given data.

#     Attributes:
#         layers: List of layers in this model.
#         built: Whether the model is built or not.
#         _build_input_shape: Shape of the input tensor.
#         loss: Loss function to use.
#         optimizer: Optimizer to use.
#         metrics: Metrics to use.
#         lr: Learning rate.
#     '''
#     def __init__(self, layers=np.array([]), name='Sequential'):
#         self.built = False
#         self.layers = layers
#         self.name = name

#         # If input shape is known, build the model
#         if len(self.layers) > 0 and layers[0].input_shape is not None:
#             self.build(self.layers[0].input_shape)

#     def add(self, layer):
#         self.layers.append(layer)

#         if self.built:
#             self.build(self._build_input_shape)
    
#     def build(self, input_shape):
#         self._build_input_shape = input_shape
#         for layer in self.layers:
#             layer.build(input_shape)
#             input_shape = layer.compute_output_shape(input_shape)
#         self.built = True
    
#     def call(self, inputs):
#         y = inputs
#         for layer in self.layers:
#             y = layer.call(y)
#         return y
    
#     def summary(self):
#         if not self.built:
#             raise ValueError('Model is not built yet')

#         print(' Model Summary')
#         for i in range(len(' Layer (type) | Output Shape | Param #')):
#             print('-', end='')
#         print()
#         print(' Layer (type) | Output Shape | Param #')
#         for i in range(len(' Layer (type) | Output Shape | Param #')):
#             print('=', end='')
#         print()
#         total_params = 0
#         for layer in self.layers:
#             output_shape = layer.compute_output_shape(self._build_input_shape)
#             param_count = 0
#             for param in layer.__dict__:
#                 if param == 'w':
#                     param_count += np.prod(layer.__dict__[param].shape)
#                 elif param == 'b':
#                     param_count += layer.__dict__[param].shape[0]
#             print(f' {layer.__class__.__name__}', end='')

#             for i in range(len(' Layer (type) ') - len(layer.__class__.__name__) - 1):
#                 print(' ', end='')

#             print(f'| {output_shape}', end='')
            
#             for i in range(len(' Output Shape ') - len(str(output_shape)) - 1):
#                 print(' ', end='')
            
#             print(f'| {param_count}', end='')

#             for i in range(len(' Param # ') - len(str(param_count)) - 1):
#                 print(' ', end='')
#             print()

#             total_params += param_count
#         print('=====================================')
#         print(f'Total params: {total_params}')

#     def compile(self, optimizer, loss, metric):
#         if optimizer not in ['sgd']:
#             raise ValueError('Optimizer not supported')
#         if loss not in ['mse', 'crossentropy', 'binary_crossentropy', 'categorical_crossentropy']:
#             raise ValueError('Loss not supported')
#         if metric not in ['accuracy', 'mse']:
#             raise ValueError('Metric not supported')

#         self.optimizer = optimizer
#         self.loss = loss
#         self.metric = metric

#     def compute_loss(self, y_true, y_prob):
#         epsilon = 1e-10 # Error term to prevent division by zero
#         if self.loss == 'mse':
#             return np.mean((y_true - y_prob) ** 2)
#         elif self.loss == 'crossentropy' or self.loss == 'binary_crossentropy':
#             return np.mean(-y_true * np.log(y_prob + epsilon) - (1 - y_true) * np.log(1 - y_prob + epsilon))
#         elif self.loss == 'categorical_crossentropy':
#             return np.mean(-np.sum(y_true * np.log(y_prob + epsilon), axis=-1))
#         else:
#             raise ValueError('Loss not supported')

#     def compute_metric(self, y_true, y_pred):
#         if self.metric == 'accuracy':
#             return np.mean(y_true == y_pred)
#         elif self.metric == 'mse':
#             return np.mean((y_true - y_pred) ** 2)
#         else:
#             raise ValueError('Metric not supported')

#     def one_hot_encode(self, y):
#         if len(y.shape) == 1:
#             y_one_hot = tf.one_hot(y, self.layers[-1].units)
#             return y_one_hot
#         else:
#             y_one_hot = np.zeros((len(y), self.layers[-1].units))
#             y_one_hot[np.arange(len(y)), y] = 1
#             return y_one_hot
    
#     def forward_propagation(self, X, y):
#         '''
#         Parameters:
#         X: Input data
#         y: Labels

#         Algorithm to propagate the input forward through the network to compute the loss and metric.
#         '''

#         X = np.array(X)
#         y = np.array(y)
        
#         # One-hot encode the labels
#         y_true = self.one_hot_encode(y)
        
#         # Forward propagation
#         y_prob = self.call(X)

#         if y_prob.shape[-1] == 1:
#             y_pred = np.array([0 if y_prob[i] > 0.5 else 1 for i in range(len(y_prob))])
#         else:
#             y_pred = np.argmax(y_prob, axis=-1)

#         # Compute loss
#         loss = self.compute_loss(y_true, y_prob)

#         # Compute metric
#         metric = self.compute_metric(y, y_pred)

#         return y_prob, y_pred, loss, metric
        
#     def backward_propagation(self, X, y, y_prob):
#         '''
#         Parameters:
#         X: Input data
#         y: Labels
#         y_prob: Output of the last layer

#         Algorithm to propagate the error backwards through the network, using the selected optimizer.
#         '''
        
#         y = np.array(y)

#         # One hot encode the labels
#         y_true = self.one_hot_encode(y)
        
#         if self.optimizer == 'sgd':
#             # Compute gradient of loss with respect to weights and biases
#             self.gradient_descent(X, y_true, y_prob)
#         else:
#             raise ValueError('Optimizer not supported')
        
#     def loss_gradient(self, y_true, y_prob):
#         if self.loss == 'mse':
#             return 2 * (y_prob - y_true)
#         elif self.loss == 'crossentropy' or self.loss == 'binary_crossentropy' or self.loss == 'categorical_crossentropy':
#             # Avoid vanishing gradients
#             y_prob = np.where(y_prob == 0, 1e-10, y_prob)
#             y_prob = np.where(y_prob == 1, 1 - 1e-10, y_prob)
            
#             return (y_prob - y_true) / (y_prob * (1 - y_prob))
#         else:
#             raise ValueError('Loss not supported')
    
#     def update_weights(self, grad_w, grad_b):
#         for i, layer in enumerate(self.layers):
#             layer.w = layer.w - self.lr * grad_w[i]
#             layer.b = layer.b - self.lr * grad_b[i]
        
#     def gradient_descent(self, X, y_true, y_prob):
#         '''
#         Parameters:
#         y_true: One-hot encoded labels
#         y_prob: Output of the last layer

#         Steps to gradient descent:
#         1. Initialize the gradients
#         2. Compute the gradient of the error w.r.t the weights of the output layer, we call this ∂C0/∂w(L)
#         3. Compute the gradient of the error w.r.t the biases of the output layer, we call this ∂C0/∂b(L) 
#         4. Compute the gradient of the error w.r.t the weights of the previous layer, we call this ∂C0/∂w(L-1)
#         5. Compute the gradient of the error w.r.t the biases of the previous layer, we call this ∂C0/∂b(L-1)
#         6. Update the weights and biases of the output layer using the gradients computed in step 2 and 3
#         7. Update the weights and biases of the previous layer using the gradients computed in step 4 and 5
#         8. Repeat steps 2 to 8 until all layers are updated 

#         Using chain rule:
#         ∂C0/∂w(L) = ∂z(L)/∂w(L) * ∂a(L)/∂z(L) * ∂C0/∂a(L)
#         ∂C0/∂b(L) = ∂z(L)/∂b(L) * ∂a(L)/∂z(L) * ∂C0/∂a(L)

#         Note:
#         ∂C0/∂w(L)     : Gradients of loss with respect to layer weights
#         ∂z(L)/∂w(L)   : Gradients of layer output with respect to layer weights
#         ∂a(L)/∂z(L)   : Gradients of layer activation with respect to layer output
#         ∂C0/∂a(L)     : Gradients of loss with respect to layer activation
#         ∂C0/∂b(L)     : Gradients of loss with respect to layer biases
#         ∂z(L)/∂b(L)   : Gradients of layer output with respect to layer biases

#         Simplified explanation:
#         To calculate how much the cost changes with respect to the weights, we need to know
#         1. How much the output changes with respect to the weights
#         2. How much the activation changes with respect to the output
#         3. How much the cost changes with respect to the activation
#         '''

#         # Initialize the gradients
#         grad_w = []
#         grad_b = []

#         # Compute the gradient of the error w.r.t the weights of the output layer
#         del_c_wrt_a = self.loss_gradient(y_true, y_prob)
#         del_a_wrt_z = self.layers[-1].activation(self.layers[-1].z, derivative=True)
#         del_z_wrt_w = X if len(self.layers) == 1 else self.layers[-2].a
#         del_c_wrt_w = np.dot(del_z_wrt_w.T, del_c_wrt_a * del_a_wrt_z)
#         grad_w.append(del_c_wrt_w)

#         # Compute the gradient of the error w.r.t the biases of the output layer
#         del_c_wrt_b = np.sum(del_a_wrt_z * del_c_wrt_a, axis=0)
#         grad_b.append(del_c_wrt_b)

#         # Loop over the layers starting from the second-to-last layer
#         for layer_idx in range(len(self.layers) - 2, -1, -1):
#             # Compute the gradient of the error w.r.t the output of the previous layer
#             del_z_wrt_a = self.layers[layer_idx + 1].w
#             del_c_wrt_a = np.dot(del_c_wrt_a * del_a_wrt_z, del_z_wrt_a.T)

#             # Compute the gradient of the error w.r.t the weights of the previous layer
#             del_a_wrt_z = self.layers[layer_idx].activation(self.layers[layer_idx].z, derivative=True)
#             del_z_wrt_w = X if layer_idx == 0 else self.layers[layer_idx-1].a
#             del_c_wrt_w = np.dot(del_z_wrt_w.T, del_c_wrt_a * del_a_wrt_z)
#             grad_w.insert(0, del_c_wrt_w)

#             # Compute the gradient of the error w.r.t the biases of the previous layer
#             del_c_wrt_b = np.sum(del_a_wrt_z * del_c_wrt_a, axis=0)
#             grad_b.insert(0, del_c_wrt_b)

#         # Update the weights and biases of the output layer
#         self.update_weights(grad_w, grad_b)

#     def fit(self, X, y, epochs=1, batch_size=32, lr=0.01, verbose=True, random_state=None, patience=None):
#         self.lr = lr
#         X = np.array(X)
#         y = np.array(y)

#         # Set the seed for reproducibility
#         if random_state is not None:
#             np.random.seed(random_state)

#         # Initialize early stopping variables
#         best_val_loss = float('inf')
#         epochs_without_improvement = 0

#         for epoch in range(epochs):
#             epoch_loss = 0
#             epoch_metric = 0
#             ctr = 0

#             print(f'Epoch {epoch+1}/{epochs}')

#             for i in range(0, len(X), batch_size):
#                 # Get batch
#                 size = min(batch_size, len(X) - i)

#                 # Forward propagation
#                 y_prob, _, loss, metric = self.forward_propagation(X[i:i+size], y[i:i+size])

#                 # Backward propagation
#                 self.backward_propagation(X[i:i+size], y[i:i+size], y_prob)

#                 # Update epoch loss and metric
#                 epoch_loss += loss
#                 epoch_metric += metric
#                 ctr += 1

#                 # Print progress bar
#                 progress = int(20 * (i + size) / len(X))
#                 progress_bar = '[' + '=' * progress + '>' + '-' * (29 - progress) + ']'
#                 if verbose:
#                     print(f'{i+size}/{len(X)} {progress_bar} - loss: {loss:.4f} - {self.metric}: {metric:.4f}', end='\r')

#             # Compute average epoch loss and metric
#             epoch_loss /= ctr
#             epoch_metric /= ctr

#             if verbose:
#                 print(f'{len(X)}/{len(X)} [==============================] - loss: {epoch_loss:.4f} - {self.metric}: {epoch_metric:.4f}')

#             # Check if validation loss improved
#             if best_val_loss - epoch_loss > 1e-4:
#                 best_val_loss = epoch_loss
#                 epochs_without_improvement = 0
#             else:
#                 epochs_without_improvement += 1

#             # Check early stopping condition
#             if patience is not None and epochs_without_improvement >= patience:
#                 print(f'Early stopping, no improvement for {patience} epochs.')
#                 break

#     def evaluate(self, X, y):
#         X = np.array(X)
#         y = np.array(y)

#         _, _, loss, metric = self.forward_propagation(X, y)

#         return loss, metric

#     def predict(self, X):
#         X = np.array(X)

#         # Forward propagation
#         y_prob = self.call(X)

#         if y_prob.shape[-1] == 1:
#             y_pred = np.array([0 if y_prob[i] > 0.5 else 1 for i in range(len(y_prob))])
#         else:
#             y_pred = np.argmax(y_prob, axis=-1)

#         return y_pred


# Train the ANN model

### First Model
Output layer: 2-neuron, softmax activation  
Loss function: Binary crossentropy

In [29]:
# Input shape is of form (num_samples, num_features)
model = Sequential([
    Dense(3, activation=relu, input_shape=(1, df.shape[1] - 1)),
    Dense(3, activation=relu),
], name='First model')

model.add(Dense(2, activation=softmax))

model.summary()

 Model Summary
--------------------------------------
 Layer (type) | Output Shape | Param #
 Dense        | (1, 3)       | 93      
 Dense        | (1, 3)       | 12      
 Dense        | (1, 2)       | 8       
Total params: 113


In [30]:
# Compile the model
model.compile(optimizer='sgd', loss='crossentropy', metric='accuracy')

In [31]:
# Train the model
model.fit(X_train, y_train, epochs=200, batch_size=32, lr=0.01, verbose=True, random_state=42, patience=5)

Epoch 1/200
455/455 [==============================] - loss: 1.0834 - accuracy: 0.7104
Epoch 2/200
455/455 [==============================] - loss: 0.3773 - accuracy: 0.8875
Epoch 3/200
455/455 [==============================] - loss: 0.3096 - accuracy: 0.9042
Epoch 4/200
455/455 [==============================] - loss: 0.2747 - accuracy: 0.9083
Epoch 5/200
455/455 [==============================] - loss: 0.2497 - accuracy: 0.9271
Epoch 6/200


455/455 [==============================] - loss: 0.2234 - accuracy: 0.9333
Epoch 7/200
455/455 [==============================] - loss: 0.1945 - accuracy: 0.9396
Epoch 8/200
455/455 [==============================] - loss: 0.1701 - accuracy: 0.9458
Epoch 9/200
455/455 [==============================] - loss: 0.1546 - accuracy: 0.9563
Epoch 10/200
455/455 [==============================] - loss: 0.1427 - accuracy: 0.9563
Epoch 11/200
455/455 [==============================] - loss: 0.1334 - accuracy: 0.9646
Epoch 12/200
455/455 [==============================] - loss: 0.1258 - accuracy: 0.9667
Epoch 13/200
455/455 [==============================] - loss: 0.1158 - accuracy: 0.9688
Epoch 14/200
455/455 [==============================] - loss: 0.1074 - accuracy: 0.9688
Epoch 15/200
455/455 [==============================] - loss: 0.1000 - accuracy: 0.9708
Epoch 16/200
455/455 [==============================] - loss: 0.0932 - accuracy: 0.9708
Epoch 17/200
455/455 [==========================

In [32]:
# Evaluate the model
from sklearn.metrics import classification_report

y_pred = model.predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.95      0.98      0.97        43
           1       0.99      0.97      0.98        71

    accuracy                           0.97       114
   macro avg       0.97      0.97      0.97       114
weighted avg       0.97      0.97      0.97       114



### Second Model
Output layer: Single neuron, sigmoid activation  
Loss function: Mean squared error

In [33]:
model2 = Sequential([
    Dense(2, activation=relu, input_shape=(1, df.shape[1] - 1)),
    Dense(3, activation=relu),
    Dense(3, activation=relu),
    Dense(1, activation=sigmoid)
], name='Second model')

model2.summary()

 Model Summary
--------------------------------------
 Layer (type) | Output Shape | Param #
 Dense        | (1, 2)       | 62      
 Dense        | (1, 3)       | 9       
 Dense        | (1, 3)       | 12      
 Dense        | (1, 1)       | 4       
Total params: 87


In [34]:
# Compile the model
model2.compile(optimizer='sgd', loss='mse', metric='accuracy')

In [35]:
# Train the model
model2.fit(X_train, y_train, epochs=200, batch_size=32, lr=0.01, verbose=True, random_state=42, patience=5)

Epoch 1/200
455/455 [==============================] - loss: 0.2265 - accuracy: 0.6301
Epoch 2/200


455/455 [==============================] - loss: 0.1534 - accuracy: 0.7729
Epoch 3/200
455/455 [==============================] - loss: 0.1209 - accuracy: 0.8292
Epoch 4/200
455/455 [==============================] - loss: 0.1086 - accuracy: 0.8479
Epoch 5/200
455/455 [==============================] - loss: 0.0951 - accuracy: 0.8812
Epoch 6/200
455/455 [==============================] - loss: 0.0813 - accuracy: 0.9104
Epoch 7/200
455/455 [==============================] - loss: 0.0720 - accuracy: 0.9271
Epoch 8/200
455/455 [==============================] - loss: 0.0601 - accuracy: 0.9437
Epoch 9/200
455/455 [==============================] - loss: 0.0468 - accuracy: 0.9646
Epoch 10/200
455/455 [==============================] - loss: 0.0425 - accuracy: 0.9688
Epoch 11/200
455/455 [==============================] - loss: 0.0398 - accuracy: 0.9688
Epoch 12/200
455/455 [==============================] - loss: 0.0378 - accuracy: 0.9708
Epoch 13/200
455/455 [==============================

In [36]:
# Evaluate the model
y_pred2 = model2.predict(X_test)

print(classification_report(y_test, y_pred2))

              precision    recall  f1-score   support

           0       0.95      0.98      0.97        43
           1       0.99      0.97      0.98        71

    accuracy                           0.97       114
   macro avg       0.97      0.97      0.97       114
weighted avg       0.97      0.97      0.97       114

